In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Please enter path to folder containing all folds (Project_Data) and output folder path where result needs to be saved

input_data_path = "./Project_Data/"
output_path = "./Prep_Data/"

In [0]:
# Read File Data and Return a list of dicts

def readFile(filepath):
  desc_list = [ "RecordID", "Age", "Gender", "Height", "ICUType", "Weight",
             "Albumin", "ALP", "ALT", "AST", "Bilirubin", "BUN", "Cholesterol",
             "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT",
             "HR", "K", "Lactate", "Mg", "MAP", "MechVent", "Na", "NIDiasABP",
             "NIMAP", "NISysABP", "PaCO2", "PaO2", "pH", "Platelets", 
             "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT",
             "Urine", "WBC"]
  f = open(filepath, 'r')
  rows = []
  for row in f.readlines():
	  	rows.append(row)
  # print(rows)

  res = {}
  # res["time"] = []
  for var in desc_list:
    res[var] = []
  for row in rows[1:]:
    # p1["time"].append(row.split(',')[0])
    res[row.split(',')[1]].append([row.split(',')[0],row.rstrip().split(',')[2]])
  
  return res


file_path = input_data_path + "Fold1/132539.txt"
f = readFile(file_path)
print(len(f))

42


In [0]:
# File data to LF Dataframe

import pandas as pd
import copy 

def getLFDF(filedata):
  rid = filedata['RecordID'][0][1]
  # print(rid)

  df = pd.DataFrame(columns=['Attr_Name', 'time', 'Attr_Value'])

  for key, value in filedata.items():
    # print(key, '->', value)

    if key == 'RecordID':
      continue
    
    if len(value) == 0:
      df.loc[len(df)] = [key, 'NA', 'NA']
      continue

    temp_val = copy.deepcopy(value)
    new_value = [[key] + x for x in temp_val]
    # print(new_value)
    df = df.append(pd.DataFrame(data=new_value, 
                                columns=['Attr_Name', 'time', 'Attr_Value']), 
                   ignore_index = True)
  
  df['rid'] = rid
  return df
    

df = getLFDF(f)
df.head()

,Attr_Name,time,Attr_Value,rid
0,Age,00:00,54,132539
1,Gender,00:00,0,132539
2,Height,00:00,-1,132539
3,ICUType,00:00,4,132539
4,Weight,00:00,-1,132539


In [0]:
# Get the list of file names

import glob

filepaths = []

for i in range(1,5):
  filepaths.extend(glob.glob(input_data_path + "Fold"+str(i)+"/*.txt"))

print(len(filepaths))

4000


In [0]:
# Get a list of LF Dataframes

dfs = []

for filepath in filepaths:
  filedata = readFile(filepath)
  dfs.append(getLFDF(filedata))

len(dfs)

4000

In [0]:
# Concat all LF dataframes into one result df

res_df = dfs[0].copy()

for i in range(1, len(dfs)):
  res_df = res_df.append(dfs[i], ignore_index = True)

res_df.shape

(1794987, 4)

In [0]:
# Save the result

res_df.to_csv(output_path + "Viz_Data.csv", index=False)